# Sprawdzanie dokładności modelu
Plik test_data.csv jest dostępny tylko dla jury. Zawiera dane testowe - jednakowe dla wszystkich drużyn. Jest w takim samym formacie co zbiór, który otrzymaliście wraz z zadaniem, tylko już z czystymi danymi.

In [1]:
import pandas as pd
import numpy as np


In [2]:
test_data = pd.read_csv("final_train.csv")

### Miejsce na twój kod:

In [3]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from time import time 
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import seaborn as sns

# Correct the order of values and label
del test_data["Unnamed: 0"]

# Calculate correlation matrix between columns
cor_matrix = test_data.corr().abs()

# Drop highest correlated columns
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
df_analysis = test_data.drop(test_data[to_drop], axis=1) 

# Divide dataframe into values and labels
Y_labels = df_analysis['Activity']
X_values = df_analysis.loc[:, df_analysis.columns != 'Activity']

# Standardising the data
# Center to the mean and component wise scale to unit variance
# for col in [variables]:
#     X_all[col] = scale(X_all[col])
sc = StandardScaler()
X_values = sc.fit_transform(X_values)

# Dividing data into training and test set taking 80% of prior data as a training set,
# so that our models are tested based on matches that took place after matches from our training set
X_train, X_test= np.split(X_values, [int(.8 *len(X_values))])
Y_train, Y_test= np.split(Y_labels, [int(.8 *len(Y_labels))])

# Training the classifier and measuring its time
def train(clf, X_train, Y_train):
    
    # Starting the clock, training the classifier, then stoping the clock
    start = time()
    clf.fit(X_train, Y_train)
    end = time()
    
    # Printing the results
    print ("Model trained in: ",end - start)
    
# Making predictions 
def predict(clf, features, target):
    
    # Starting the clock, making predictions, then stoping the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Printing results
    print ("Prediction made in: ",end - start)
    
    # Returning F1 score and accuracy
    return f1_score(target, y_pred, average=None), sum(target == y_pred) / float(len(y_pred)), y_pred

# Evaluation of our model
def fit(clf, X_train, y_train, X_test, y_test):
    
    # Indicating the model and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Training model
    train(clf, X_train, y_train)
    
    # Printing the results of prediction for both training and testing set
    f1, acc, output = predict(clf, X_train, y_train)
    print ("===========================================")
    print ("Indexes assesing methods for training set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)
    
    print ("===========================================")
    f1, acc, output = predict(clf, X_test, y_test)
    print ("Indexes assesing methods for test set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)
    
# Creating our model and training it
clf_XGB = xgb.XGBClassifier()

fit(clf_XGB, X_train, Y_train, X_test, Y_test)

f1f, accf, predicted = predict(clf_XGB, X_values, Y_labels)

Training a XGBClassifier using a training set size of 8868. . .


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:11:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Model trained in:  27.490998029708862
Prediction made in:  0.04799818992614746
Indexes assesing methods for training set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1. 1. 1. 1. 1. 1.]
Accuracy: 1.0
Prediction made in:  0.014999151229858398
Indexes assesing methods for test set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1.         0.98701299 0.9867374  0.99337748 0.98686679 0.99298738]
Accuracy: 0.9914337240757439
Prediction made in:  0.059999942779541016


### Sprawdzanie dokładności:
Predicted - zmienna przechowująca przewidzianą aktywność. Poniższa linijka powinna zwrócić wartość 0-1.

In [4]:
np.mean(predicted == test_data.Activity)

0.9982861266462204